In [1]:
import numpy as np
import pandas as pd

import random

import sys
sys.path.append("../")

from src.object import *
from src.create import *

from config.formation import *

In [2]:
df_name = pd.read_csv("../data/csv/NationalNames.csv")
df_name_list = list(df_name[df_name.Gender=='M'].Name.sample(500))

In [3]:
df_place = pd.read_csv("../data/csv/results.csv")
df_place = df_place.city.unique()

In [4]:
class ProSoccerLeague:
    def __init__(self, name, leagues):
        self.name = name
        self.leagues = leagues
        self.competition_result = {}
    
    def play_1season(self, year):
        for league in self.leagues:
            league.play_1year(year)
        
        for index in range(len(self.leagues)):
            season_name = f"{self.leagues[index].name}_{year}"
            if self.leagues[index].category!="top":
                promotion = self.leagues[index].promotion[season_name]
                promotion_team = [t for t in self.leagues[index].teams if t.name in promotion]
                self.leagues[index].teams = [s for s in self.leagues[index].teams if s not in promotion_team]
                self.leagues[index-1].teams.extend(promotion_team)

            if self.leagues[index].category!="lowest":
                relegation = self.leagues[index].relegation[season_name]
                relegation_team = [t for t in self.leagues[index].teams if t.name in relegation]
                self.leagues[index].teams = [s for s in self.leagues[index].teams if s not in relegation_team]
                self.leagues[index+1].teams.extend(relegation_team)
        
        for l in self.leagues:
            l.set_team_leaguename()
    
    def play_1competition(self, competition_name, year, kind):
        join_teams = []
        for l in self.leagues:
            join_teams.extend(l.teams)
            l.set_player_result(competition_name, year, kind)

        for max_round in range(N):
            if len(join_teams) < pow(2, max_round):
                break

        join_team = random.shuffle(join_teams)

        result = pd.DataFrame(columns=["チームA", "チームB", "勝利", "スコア", "ラウンド"])
        round_ = 1

        while True:
            buff_teams = join_teams.copy()
            for i in range(0, len(join_teams), 2):
                game_team = join_teams[i:i+2]
                if len(game_team) < 2:
                    continue

                cup_game = Game(home=game_team[0], 
                                away=game_team[1], 
                                competition_name=competition_name,
                                pk=1)
                cup_game.battle()
                if cup_game.result=="home" or cup_game.result=="home-pk":
                    win = game_team[0]
                    lose = game_team[1]
                else:
                    win = game_team[1]
                    lose = game_team[0]

                if max_round-3 >= round_:
                    round_name = f"{round_}回戦"
                    result_name = f"{round_}回戦"
                elif max_round-2 == round_:
                    round_name = f"準々決勝"
                    result_name = f"ベスト8"
                elif max_round-1 == round_:
                    round_name = f"準決勝"
                    result_name = f"ベスト4"
                elif max_round == round_:
                    round_name = f"決勝"
                    result_name = f"準優勝"
                
                if "pk" in cup_game.result:
                    score = f"{cup_game.home_goal}-{cup_game.away_goal}(pk:{cup_game.home_pk_goal}-{cup_game.away_pk_goal})"
                else:
                    score = f"{cup_game.home_goal}-{cup_game.away_goal}"

                output = pd.Series([f"{game_team[0].name}({game_team[0].league_name})", 
                                    f"{game_team[1].name}({game_team[1].league_name})",
                                    win.name, score, round_name], 
                                   index=["チームA", "チームB", "勝利", "スコア", "ラウンド"])
                result.loc[f"{game_team[0].name}-{game_team[1].name}"] = output
                
                
                lose.competition_result[competition_name] = result_name
                buff_teams.remove(lose)
            round_ += 1
            join_teams = buff_teams
            if len(join_teams)<2:
                win.competition_result[competition_name] = "優勝"
                break
        
        self.competition_result[competition_name] = result

In [5]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(75, 80)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i], 
             formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L1 = League(name="premia", 
            teams=league_team,
            num=20,
            category='top',
            relegation_num=3)

In [6]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(75, 80)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+20], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L2 = League(name="championship",
            teams=league_team,
            num=20,
            category='middle',
            relegation_num=3,
            promotion_num=3)

In [7]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(75, 80)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+40], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L3 = League(name="EFL1", 
            teams=league_team,
            num=20,
            category='middle',
            relegation_num=3,
            promotion_num=3)

In [8]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(75, 80)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+60], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L4 = League(name="EFL2", 
            teams=league_team,
            num=20,
            category='lowest',
            relegation_num=0,
            promotion_num=3)

In [9]:
England = ProSoccerLeague(name="england", leagues=[L1, L2, L3, L4])
for year in range(1990, 2020):
    print(year)
    England.play_1competition(f"FA杯_{year}", year, "カップ戦")
    England.play_1season(year)

1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [29]:
league_num = 1
team_num = 0

print([t.name for t in England.leagues[league_num].teams])
print()
print(England.leagues[league_num].champion)
print()

print(England.leagues[league_num].promotion)
print()
print(England.leagues[league_num].relegation)


['London', 'Bristol', 'Wrexham', 'Liège', 'Newark', 'Manchester', 'Saint-Ouen', 'Stuttgart', 'Liverpool', 'Blackburn', 'Prague', 'Maisons-Alfort', 'Bradford', 'Shrewsbury', 'Greenock', 'Rotterdam', 'Karlsruhe', 'Portsmouth', 'Kristiania', 'Glasgow']

{'Brussels': ['championship_1990'], 'Middlesbrough': ['championship_1991'], 'Kleve': ['championship_1992'], 'Luxembourg': ['championship_1993'], 'Aberdeen': ['championship_1994'], 'La Chaux-de-Fonds': ['championship_1995', 'championship_2003', 'championship_2014', 'championship_2017'], 'Edinburgh': ['championship_1996'], 'Solna': ['championship_1997', 'championship_2007'], 'Saint-Ouen': ['championship_1998'], 'Geneva': ['championship_1999', 'championship_2004', 'championship_2016'], 'Blackburn': ['championship_2000'], 'Uccle': ['championship_2001', 'championship_2019'], 'Greenock': ['championship_2002', 'championship_2009'], 'Richmond': ['championship_2005'], 'Karlsruhe': ['championship_2006'], 'Maisons-Alfort': ['championship_2008', 'cham

In [31]:
team_num = 15
print(England.leagues[league_num].teams[team_num].name)
print(England.leagues[league_num].teams[team_num].competition_result)
England.leagues[league_num].teams[team_num].result

Rotterdam
{'FA杯_1990': '準優勝', 'FA杯_1991': '1回戦', 'FA杯_1992': '2回戦', 'FA杯_1993': '1回戦', 'FA杯_1994': '2回戦', 'FA杯_1995': '3回戦', 'FA杯_1996': '2回戦', 'FA杯_1997': '1回戦', 'FA杯_1998': '1回戦', 'FA杯_1999': '1回戦', 'FA杯_2000': '2回戦', 'FA杯_2001': '2回戦', 'FA杯_2002': 'ベスト8', 'FA杯_2003': '2回戦', 'FA杯_2004': '2回戦', 'FA杯_2005': '1回戦', 'FA杯_2006': '1回戦', 'FA杯_2007': '2回戦', 'FA杯_2008': '2回戦', 'FA杯_2009': '2回戦', 'FA杯_2010': '1回戦', 'FA杯_2011': '3回戦', 'FA杯_2012': '2回戦', 'FA杯_2013': '2回戦', 'FA杯_2014': '3回戦', 'FA杯_2015': '1回戦', 'FA杯_2016': '2回戦', 'FA杯_2017': '3回戦', 'FA杯_2018': '3回戦', 'FA杯_2019': '4回戦'}


,win,lose,row,得点,失点,得失点差,Points,順位,リーグ名
EFL1_1990,17,11,10,54,32,22,61,6位,EFL1
EFL1_1991,17,9,12,54,33,21,63,8位,EFL1
EFL1_1992,16,10,12,43,32,11,60,6位,EFL1
EFL1_1993,17,6,15,46,21,25,66,7位,EFL1
EFL1_1994,22,8,8,50,26,24,74,3位,EFL1
championship_1995,15,11,12,37,28,9,57,7位,championship
championship_1996,14,12,12,28,28,0,54,9位,championship
championship_1997,10,14,14,30,34,-4,44,16位,championship
championship_1998,14,13,11,44,35,9,53,10位,championship
championship_1999,14,17,7,36,42,-6,49,14位,championship


In [23]:
for p in England.leagues[league_num].teams[team_num].affilation_players:
    if p.partification_position=="ST":
        break
print(p.name, p.main_position, p.partification_position)
df = pd.DataFrame(p.result.values(), index=p.result.keys())
df_cup = df[df["分類"]=="カップ戦"]
df_league = df[df["分類"]=="リーグ"]
df_league.sort_values("年度")

Darrly ST ST


,goal,assist,年度,分類
EFL2_1990,8,1,1990,リーグ
EFL2_1991,4,1,1991,リーグ
EFL2_1992,6,2,1992,リーグ
EFL2_1993,6,2,1993,リーグ
EFL2_1994,9,1,1994,リーグ
EFL2_1995,6,0,1995,リーグ
EFL2_1996,5,1,1996,リーグ
EFL2_1997,5,3,1997,リーグ
EFL2_1998,8,2,1998,リーグ
EFL2_1999,9,3,1999,リーグ


In [26]:
England.competition_result["FA杯_1990"]

,チームA,チームB,勝利,スコア,ラウンド
Greenock-Liège,Greenock(championship),Liège(EFL2),Greenock,0-0(pk:14-13),1回戦
Southampton-Maisons-Alfort,Southampton(championship),Maisons-Alfort(EFL2),Southampton,0-0(pk:5-4),1回戦
Zwolle-Wrexham,Zwolle(EFL2),Wrexham(premia),Wrexham,0-3,1回戦
Derby-Karlsruhe,Derby(championship),Karlsruhe(EFL1),Derby,2-0,1回戦
Aberdare-Nottingham,Aberdare(EFL1),Nottingham(championship),Aberdare,3-0,1回戦
...,...,...,...,...,...
Rotterdam-Kleve,Rotterdam(EFL1),Kleve(EFL2),Rotterdam,2-1,4回戦
Sheffield-Sunderland,Sheffield(premia),Sunderland(premia),Sunderland,1-2,準々決勝
Bangor-Manchester,Bangor(premia),Manchester(premia),Bangor,3-0,準々決勝
Sunderland-Bangor,Sunderland(premia),Bangor(premia),Bangor,0-1,準決勝
